In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import urllib
import urllib.request as req
import pandas as pd

In [22]:
# search_word = input('검색어를 입력해주세요')
search_word='NFT'
search_word =urllib.parse.quote(search_word)
url='http://www.yes24.com/Product/Search?domain=ALL&query=%s'%search_word

driver = webdriver.Chrome('tools/chromedriver')
driver.implicitly_wait(3)

page_num=2
total=1
driver.get(url)

result=[]

#goodsListWrap > div.sGoodsPagen > div > a:nth-child(10)
while True:
    if page_num ==11:
        time.sleep(1)
        next = driver.find_element(By.CSS_SELECTOR,'#goodsListWrap > div.sGoodsPagen > div > a.bgYUI.next')
        next.click()
        page_num=4
        time.sleep(1)

    html = driver.page_source
    soup = bs(html,'html.parser')
    book_list = soup.select('#yesSchList>li')
    for book in book_list:
        title = book.select_one('.gd_name').string
        img= book.select_one('div.item_img > div.img_canvas > span > span > a > em > img')
        if 'data-original' in img.attrs.keys():
            img_url= img['data-original']
            # save_image(img_url,total)
        else:
            img_url=img['src']
            # save_image(img_url,total)
            
        author_list = book.select('.info_auth>a')
        if author_list: 
            author=','.join([author.string for author in author_list])
            pub=book.select_one('.info_pub>a').string
            date = book.select_one('.info_date').string
        else:
            author,pub,date = ("", "", "")
        price = book.select_one('.info_price .txt_num em').string
        grade = book.select_one('.rating_grade em')
        grade = grade.string if grade else ""
        
        result.append((title,author,pub,date,price,grade))
        total+=1
            
    try:
        next_num = driver.find_element(By.CSS_SELECTOR,'#goodsListWrap > div.sGoodsPagen > div > a:nth-child(%s)'%page_num)
        next_num.click()
        time.sleep(1)
        page_num+=1
    except:
        print('page end :: total %s'%total)
        break
               
    
driver.close()

C:\Users\LGuser\AppData\Local\Temp\ipykernel_13080\2506401395.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('tools/chromedriver')


page end :: total 320


In [15]:
def save_image(url,idx):
    save_name='data/book_images/image'+str(idx)+'.jpg'
    req.urlretrieve(url,save_name)
    print(save_name,'이(가) 저장되었습니다')

In [51]:
df = pd.DataFrame(result)
result
df.columns=['title', 'author', 'pub', 'date', 'price', 'grade']
df.to_csv('books2.csv', index=False)
df

,title,author,pub,date,price,grade
0,NFT 레볼루션,"성소라,롤프 회퍼,스콧 맥러플린",더퀘스트,2021년 09월,"16,200",9.3
1,세븐 테크,"김미경,김상균,김세규,김승주,정지훈",웅진지식하우스,2022년 02월,"16,200",9.6
2,NFT 사용설명서,"맷 포트나우,큐해리슨 테리,남경보,이장우",여의도책방,2021년 11월,"16,200",9.5
3,"NFT, 처음 만나는 세계","심상용,디사이퍼,캐슬린 김,이민하,김성혜",시공아트,2022년 05월,"14,400",None
4,한 걸음씩 따라 하는 NFT 아트,루미블루,루비페이퍼,2022년 06월,"16,200",None
...,...,...,...,...,...,...
314,METAVERSE INVESTING 2022,None,None,None,"35,360",None
315,Strategische Markenfuhrung Fur Konsumguter. De...,None,None,None,"70,730",None
316,China - Weltwirtschaftsmacht der Zukunft?,None,None,None,"19,700",None
317,"""Chinindia"" - Futurologie oder nahe Zukunft?",None,None,None,"22,530",None


In [21]:
import pymysql
result

[('NFT 레볼루션', '성소라,롤프 회퍼,스콧 맥러플린', '더퀘스트', '2021년 09월', '16,200', '9.3'),
 ('세븐 테크', '김미경,김상균,김세규,김승주,정지훈', '웅진지식하우스', '2022년 02월', '16,200', '9.6'),
 ('NFT 사용설명서 ',
  '맷 포트나우,큐해리슨 테리,남경보,이장우',
  '여의도책방',
  '2021년 11월',
  '16,200',
  '9.5'),
 ('NFT, 처음 만나는 세계',
  '심상용,디사이퍼,캐슬린 김,이민하,김성혜',
  '시공아트',
  '2022년 05월',
  '14,400',
  None),
 ('한 걸음씩 따라 하는 NFT 아트', '루미블루', '루비페이퍼', '2022년 06월', '16,200', None),
 ('쉽게 따라하는 NFT 마스터 가이드',
  '티아나 로런스,김서영,정종화',
  '한빛미디어',
  '2022년 06월',
  '18,000',
  None),
 ('나의 첫 NFT 교과서', '황성배,전래훈', '페이지2', '2022년 05월', '15,300', None),
 ('NFT 미래수업', '홍기훈', '한국경제신문사(한경비피)', '2022년 03월', '16,200', '9.5'),
 ('나의 첫 NFT 투자 수업', '황정환,김현호,박지영,강민주', '클랩북스', '2022년 06월', '16,200', '10.0'),
 ('NFT 아트 실전 수업', '돈신궁예', '라이온북스', '2022년 04월', '15,120', '10.0'),
 ('NFT는 처음입니다', '김일동', '세종서적', '2022년 01월', '15,300', '10.0'),
 ('암호화폐와 NFT, 무엇이 문제일까?', '김승주', '동아엠앤비', '2022년 03월', '13,500', '9.2'),
 ('비트코인에 가려진 세상, 이더리움 ', '코인 트레이너', '지식오름', '2022년 02월', '16,200', '9.3'),
 ('NFT 

In [26]:
list= result[:3]
print(list)
def insert_book(list):
    conn=pymysql.connect(
        host='localhost',
        user='kej',
        password='1234',
        db='pydb',
        charset='utf8')
    cursor=conn.cursor()
    sql='insert into book(title, author, pub, date, price, grade) values(%s,%s,%s,%s,%s,%s)'
    cursor.executemany(sql,list)
    conn.commit()
    conn.close()
insert_book(list)

[('NFT 레볼루션', '성소라,롤프 회퍼,스콧 맥러플린', '더퀘스트', '2021년 09월', '16,200', '9.3'), ('세븐 테크', '김미경,김상균,김세규,김승주,정지훈', '웅진지식하우스', '2022년 02월', '16,200', '9.6'), ('NFT 사용설명서 ', '맷 포트나우,큐해리슨 테리,남경보,이장우', '여의도책방', '2021년 11월', '16,200', '9.5')]


OperationalError: (1205, 'Lock wait timeout exceeded; try restarting transaction')